<font size="5">Mean First Passage Time</font>

In [4]:
import numpy as np
import os

os.makedirs('./Analysis',exist_ok=True)

def move_row_col(matrix, i):
    tmp = np.delete(matrix, i, axis=0)
    move_matrix = np.delete(tmp, i, axis=1)
    return move_matrix


def mfpt(tpm,lag_time,num_state):
    """
    Calculate mfpt from a N*N transition probability matrix 

    Parameters
    ----------
    tpm: ndarray
        a N*N transition probability matrix
    lag_time: int
        The lag time for outputing TPM. 
        The MPFT calculated will share the same unit as lag time.
    num_state: int
        Number of macrostate in your model
        
    Return
    ----------
    mfpt: ndarray
        a num_state*num_state ndarray containing MFPT, 
        which the number at the i-th row and j-th column 
        represent MFPT from i-th state to j-th state
    
    References
    ----------
    ..[1] Sheskin, J.T.(1993). Computing mean first passage times for a Markov chain.
           Int J Math Educ Sci Technol. 30:2.167-185.

    """
    mfpt_a = np.zeros((num_state*(num_state-1), num_state*(num_state-1)))
    mfpt_b = np.ones(num_state*(num_state-1))
    for j in range(num_state):
        index_ini = j*(num_state-1)
        index_end = index_ini + num_state -1
        mfpt_a[index_ini:index_end, index_ini:index_end] = move_row_col(tpm, j)
    mfpt_a -= np.identity(num_state*(num_state-1))
    mfpt_b =np.ones(num_state*(num_state-1)) * -lag_time
    mfpt = np.linalg.solve(mfpt_a, mfpt_b)
    n=0
    for i in range(num_state):
        mfpt=np.insert(mfpt,n,0)
        n+=int(num_state+1)
    mfpt=mfpt.reshape(num_state,num_state)
    print(mfpt)
    return mfpt

TPM=np.loadtxt('./qMSM/qMSM_Propagate_TPM.txt')
num_state = 4
TPM = np.reshape(TPM, (len(TPM), num_state, num_state))
lagtime=250

sample_mfpt=mfpt(TPM[30],lag_time=30,num_state=4)




[[     0.          77682.37314859  80741.38912361  79545.43679093]
 [  2145.62154571      0.           3154.96462478   2198.71313453]
 [ 19582.28174948  17532.60885353      0.          17557.74110905]
 [  2274.24239836    464.27034484   1445.65409062      0.        ]]


<font size="5">Macrostate Sampling</font>

In [ ]:
import numpy as np
import mdtraj as md
import sys
import optparse
import linecache
from glob import glob
import re 

def sample_macrostate(trajDir="./trajs/",topfile = "./trajs/npt_fit.pdb",
                      lumped_assignment='./qMSM/lumping_assignment.npy',
                      analysis_dir='./Analysis/',no_of_sample=10):
    """
    Sample structures from macrostate randomly and save as PDB files. 
    A file containing the origin of the conformation will also be generated
    
    Parameters
    ----------
    trajDir : str, Default="./trajs/"
        Directory containing the MD trajectories
        
    topfile : str, Default=trajDir+"npt_fit.pdb"
        Directory containing the topology for  MD trajectories
        
    lumped_assignment: str, Default='./qMSM/lumping_assignment.npy'
        Directory containing the macrostate assignment
    
    analysis_dir: str , Default='./analysis/',
        Directory for saving the output 
    
    no_of_sample: int, Deafult=10
        Number of structures sampled from each macrostate
    
    """
    trajlist=glob(trajDir+"*.xtc")
    trajlist.sort(key=lambda f: int(re.sub('\D', '', f)))

    macro_assignment=np.load(lumped_assignment)
    macro_assignment=np.reshape(macro_assignment,(84,5001))
    states=np.unique(macro_assignment)

    out=open("{}sampled_pdb_list.txt".format(analysis_dir),"wt")
    out.write("macro_state\toutput_filename\ttraj_index\tframe_index\n")
    for i in states:
        b=np.argwhere(macro_assignment==i)
        random_index=np.random.choice(b.shape[0],size=no_of_sample)
        b=b[random_index]
        for j,n in enumerate(b):
            output_filename=analysis_dir+"macro%d_sample%d.pdb" % (i,j)
            out.write(("%f\t%s\t%s\t%d\n" % (i, str(output_filename), trajlist[n[0]],  n[1])))
            traj = md.load_frame(trajlist[n[0]], n[1] , top=topfile)
            traj.save_pdb(output_filename)

        
analysis_dir='./Analysis/'           
trajDir="./trajs/"
topfile = trajDir+"npt_fit.pdb"
lumped_assignment='./qMSM/lumping_assignment.npy'
no_of_sample=10
sample_macrostate(trajDir,topfile,lumped_assignment,analysis_dir,no_of_sample)